# Using Tractor catalog for positions

In [2]:
import matplotlib.pyplot as plt
from astropy.io import fits as fits
from astropy.wcs import WCS
from astropy.utils import data as data_utils
from astropy.nddata import bitmask

# version dependencies are currently an issue
import pkg_resources
pkg_resources.require("numpy==1.26.3")
import numpy as np

# import json
# import requests
from dl import authClient as ac, queryClient as qc
import pandas as pd

## Image query

In [25]:
def query_tractor(ra_b, dec_b, dist):
    """Queries the Astro Data Lab for the ra, dec and mag_g of the objects within a square of side length (dist).     
    dist is in degrees
    """
    #Bounds of the square we are querying objects for
    ra_min=ra_b
    ra_max = ra_b + dist
    dec_min=dec_b
    dec_max = dec_b + dist

    query = f"""
    SELECT ra, dec, mag_g
    FROM ls_dr10.tractor_s
    WHERE ra >= ({ra_min}) AND ra < ({ra_max})
    AND dec >= ({dec_min}) AND dec < ({dec_max})
    AND mag_g>=16 AND mag_g<=21
    """

    print(query)
    
    print("Querying the Astro Data Lab...")
    # check if this completes successfuly
    brick_info = qc.query(sql=query, fmt="pandas")
    return brick_info

#tractor_df = query_tractor(100, -25, 1.0)

In [26]:
for ra_b in [0,1]:
    for dec_b in [-90,-89]:
        info=query_tractor(ra_b,dec_b,dist=1)
        print(info)


    SELECT ra, dec, mag_g
    FROM ls_dr10.tractor_s
    WHERE ra >= (0) AND ra < (1)
    AND dec >= (-90) AND dec < (-89)
    AND mag_g>=16 AND mag_g<=21
    
Querying the Astro Data Lab...
          ra        dec      mag_g
0   0.988276 -89.132644  20.988073
1   0.222668 -89.305062  20.794958
2   0.789421 -89.294624  20.364925
3   0.495449 -89.290410  20.342780
4   0.757246 -89.288594  18.739302
5   0.695922 -89.287252  19.293839
6   0.388128 -89.274055  19.988960
7   0.206767 -89.269689  18.334524
8   0.771046 -89.258430  18.900785
9   0.846243 -89.257052  16.489193
10  0.268254 -89.219230  17.323652
11  0.610288 -89.214781  17.337160
12  0.543179 -89.213696  20.522438
13  0.533915 -89.195856  20.442114
14  0.304003 -89.170935  20.236864
15  0.460989 -89.167663  18.848656
16  0.632125 -89.161085  19.964457
17  0.488745 -89.157037  18.816286
18  0.929419 -89.143405  20.007538
19  0.900438 -89.141254  19.416662
20  0.079836 -89.130658  17.743507
21  0.063236 -89.130324  17.522781
22 

## Get bitmask info

In [6]:
# there will be a text file to read from at some point lol
masked_stars = np.genfromtxt("mask_data.txt", delimiter=' ', names=True, dtype=None)
masked_stars

array([0.04868477, 0.102481  , 0.11692324, 0.00341858, 0.04326776])

## Segmentation map